In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [6]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())['col_name']
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))

createDummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())['col_name']
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

dummy_theory_output = torch.var(dummy_data_tensor, correction = 1)
dummy_data_mean = torch.mean(dummy_data_tensor)
theory_output = torch.var(data_tensor, correction = 1)
data_mean = torch.mean(data_tensor)

In [7]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*(X.size()[1]-1))<=torch.abs(0.01*self.w*(X.size()[1]-1)),x_mean_cons ),self.w)

verifier_define_calculation(dummy_data_path, ['col_name'],sel_dummy_data_path,verifier_model, verifier_model_path)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [8]:
# Prover/ data owner side
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*(X.size()[1]-1))<=torch.abs(0.01*self.w*(X.size()[1]-1)),x_mean_cons ),self.w)


prover_gen_settings(data_path,['col_name'], sel_data_path, prover_model,prover_model_path, [2], "resources", settings_path)

theory output:  tensor(212.5777)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-20332,8],"logrows":15,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1521,"total_const_size":8,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 3.455188035964966 seconds
Theory output:  tensor(212.5777)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 212.5
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14958435514946940528, 17320992765071788231, 18318297740500540397, 2188255106962760522]]], 'proof': '0b562aca073a5a971ff0dd678691c855b68e64d5e16cdc634b34cbacf9d45fac2d1acee07c3f84bb6a6eee25f3ef86fe8e219436a6436491610a2d8219133841205b101dd435bd36cd07a8964959e81da2e6cbe544bb10c3a091dc76389928002db35b4ba178f74f615698252c252b4d93355897b1935fc344af2987cc7e67b41e5a3d0f076c1f8f0298fc2be6e20c285a053bd816aa767d3d612fc0acfa49042e6ed950ad63e0b9dcbb8b94a97077ddb25d3c33ef73b20fe1fac08375f9a0c323d9eda4316a2ffdaaf189d1b32c52c81bdc6d29fdd5871d81b1433418849f7613a74ef2de4464cf7165bab063a0c973e44d6affb151ee46ea19a44b5ea0c7612d834714ddc775d697c5417dfc165ef6070aaa9a1bf77250478112264f88fa1c21b219b804d30420f4b9b2ad67a1151e6293343e3776383ad5715446ef4591040de90462a7589a0ad1c556c55152576fbc3f68aeabbd82fbd128e64e77d4

In [10]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14958435514946940528, 17320992765071788231, 18318297740500540397, 2188255106962760522]]]
proof boolean:  1.0
proof result 1 : 212.5
verified
